In [103]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import numpy as np

In [56]:
project_df = pd.read_excel("C:/Users/timsc/Desktop/MDA Kedro/horizon-funding/data/01_raw/project.xlsx")
organization_df = pd.read_excel("C:/Users/timsc/Desktop/MDA Kedro/horizon-funding/data/01_raw/organization.xlsx")



c:\Users\timsc\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\timsc\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [76]:
project_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15341 entries, 0 to 15340
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 15341 non-null  int64 
 1   title              15341 non-null  object
 2   ecMaxContribution  15341 non-null  object
 3   totalCost          15341 non-null  object
 4   startDate          15341 non-null  object
 5   endDate            15341 non-null  object
 6   fundingScheme      15340 non-null  object
dtypes: int64(1), object(6)
memory usage: 839.1+ KB


In [75]:
project_df = project_df[['id', 'title', 'ecMaxContribution', 'totalCost', 'startDate', 'endDate', 'fundingScheme']]
organization_df = organization_df[['projectID', 'organisationID', 'country', 'activityType', 'SME', 'order']]

In [198]:
organization_df.head()

,projectID,organisationID,country,activityType,SME,order
0,101116741,999981634,NL,HES,False,1
1,101163161,999997736,DK,HES,False,2
2,101163161,999854855,DE,HES,False,3
3,101163161,999990267,DE,REC,False,1
4,101160499,999874546,ES,HES,False,1


In [77]:
org_counts = (
    organization_df
    .groupby("projectID")["organisationID"]
    .nunique()
    .rename("org_count")
    .reset_index()
)

In [78]:
org_country = (
    organization_df
    .loc[organization_df["order"] == 1, ["projectID", "country"]]
    .drop_duplicates(subset="projectID")
    .rename(columns={"country": "organiser_country"})
)

In [79]:
org_country.head()

,projectID,organiser_country
0,101116741,NL
3,101163161,DE
4,101160499,ES
7,101166905,DE
8,101162875,IT


In [80]:
processed_data = pd.merge(project_df, organization_df, left_on='id', right_on='projectID')


In [ ]:
processed_data = (
    processed_data
    .merge(org_counts, left_on="id", right_on="projectID", how="left")
    .merge(org_country, left_on="id", right_on="projectID", how="left")
    .drop(columns=["projectID_x", "projectID_y"])
    .set_index('projectID')   
)


In [86]:
processed_data.head()

,ecMaxContribution,totalCost,startDate,endDate,fundingScheme,SME,order,org_count,organiser_country
projectID,,,,,,,,,
101116741,1499998,1499998,2024-02-01,2029-01-31,HORIZON-ERC,False,1,1,NL
101163161,12085363,12085363,2025-08-01,2031-07-31,HORIZON-ERC-SYG,False,2,3,DE
101163161,12085363,12085363,2025-08-01,2031-07-31,HORIZON-ERC-SYG,False,3,3,DE
101163161,12085363,12085363,2025-08-01,2031-07-31,HORIZON-ERC-SYG,False,1,3,DE
101160499,1489128,1489128,2025-06-01,2030-05-31,HORIZON-ERC,False,1,1,ES


In [83]:
processed_data = processed_data.drop(columns=["title", "organisationID", "activityType", 'id', "country"])

In [197]:
processed_data.head()

,ecMaxContribution,totalCost,startDate,endDate,fundingScheme,SME,order,org_count,organiser_country
projectID,,,,,,,,,
101116741,1499998,1499998,2024-02-01,2029-01-31,HORIZON-ERC,False,1,1,NL
101163161,12085363,12085363,2025-08-01,2031-07-31,HORIZON-ERC-SYG,False,2,3,DE
101163161,12085363,12085363,2025-08-01,2031-07-31,HORIZON-ERC-SYG,False,3,3,DE
101163161,12085363,12085363,2025-08-01,2031-07-31,HORIZON-ERC-SYG,False,1,3,DE
101160499,1489128,1489128,2025-06-01,2030-05-31,HORIZON-ERC,False,1,1,ES


In [ ]:
processed_unique = (
    processed_data
    .sort_values("order")                          
    .groupby("projectID", as_index=True)
    .first()                                      
)

In [165]:
processed_unique.head()

,ecMaxContribution,totalCost,startDate,endDate,fundingScheme,SME,order,org_count,organiser_country
projectID,,,,,,,,,
101039048,1496205,1496205,2022-02-01,2027-01-31,HORIZON-ERC,False,1,1,ES
101039060,1499150,1499150,2023-09-01,2028-08-31,HORIZON-ERC,False,1,3,ES
101039066,1482050,1482050,2023-01-01,2027-12-31,HORIZON-ERC,False,1,1,PL
101039090,1495000,1495000,2022-04-01,2027-03-31,HORIZON-AG,False,1,1,DE
101039098,1458688,1458688,2023-03-01,2028-02-29,HORIZON-ERC,False,1,1,DE


In [166]:
processed_unique = processed_unique.drop(columns=["order"])

In [167]:

processed_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15341 entries, 101039048 to 190199874
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ecMaxContribution  15341 non-null  object
 1   totalCost          15341 non-null  object
 2   startDate          15341 non-null  object
 3   endDate            15341 non-null  object
 4   fundingScheme      15340 non-null  object
 5   SME                15341 non-null  object
 6   org_count          15341 non-null  int64 
 7   organiser_country  15341 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.1+ MB


In [168]:
def cast_types(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    for col in ["ecMaxContribution", "totalCost"]:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(",", ".", regex=False)      
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")
   
    df["startDate"] = pd.to_datetime(df["startDate"], errors="coerce")
    df["endDate"]   = pd.to_datetime(df["endDate"], errors="coerce")
   
    df["startmonth"] = df["startDate"].dt.month
    return df


In [169]:
processed_unique = cast_types(processed_unique)

In [170]:
processed_unique.head(5)

,ecMaxContribution,totalCost,startDate,endDate,fundingScheme,SME,org_count,organiser_country,startmonth
projectID,,,,,,,,,
101039048,1496205.0,1496205.0,2022-02-01,2027-01-31,HORIZON-ERC,False,1,ES,2
101039060,1499150.0,1499150.0,2023-09-01,2028-08-31,HORIZON-ERC,False,3,ES,9
101039066,1482050.0,1482050.0,2023-01-01,2027-12-31,HORIZON-ERC,False,1,PL,1
101039090,1495000.0,1495000.0,2022-04-01,2027-03-31,HORIZON-AG,False,1,DE,4
101039098,1458688.0,1458688.0,2023-03-01,2028-02-29,HORIZON-ERC,False,1,DE,3


In [ ]:


def add_duration(df: pd.DataFrame) -> pd.DataFrame:
   
    df = df.copy()
    # Ensure datetime dtype
    df["startDate"] = pd.to_datetime(df["startDate"], errors="coerce")
    df["endDate"]   = pd.to_datetime(df["endDate"],   errors="coerce")
    # Compute duration
    df["duration_days"] = (df["endDate"] - df["startDate"]).dt.days
    df = df.drop(columns=["startDate", "endDate"])
    return df


In [172]:
processed_unique = add_duration(processed_unique)

In [ ]:
def drop_zero_totalcost(df):

    df = df.copy()
    return df[df['totalCost'] != 0]


In [174]:
processed_unique = drop_zero_totalcost(processed_unique)


In [ ]:
def bin_ecMaxContribution(df):
    
    df = df.copy()
   
    bins = [0,2000000,4000000,np.inf]
   
    df["funding_class"] = pd.cut(
        df["ecMaxContribution"],
        bins=bins,
        labels=['low', 'medium', 'high'],
        include_lowest=True,
        duplicates='drop' 
    )
    return df

In [176]:
processed_unique = bin_ecMaxContribution(processed_unique)


In [177]:
summary = processed_unique.groupby('funding_class')['ecMaxContribution'].agg(['min', 'max', 'count'])
print(summary)


                      min          max  count
funding_class                                
low              50000.00    2000000.0   3230
medium         2000560.00    4000000.0   2948
high           4000206.25  549442000.0   2599


C:\Users\timsc\AppData\Local\Temp\ipykernel_28904\3376535741.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = processed_unique.groupby('funding_class')['ecMaxContribution'].agg(['min', 'max', 'count'])


In [178]:
processed_unique.head(2000)

,ecMaxContribution,totalCost,fundingScheme,SME,org_count,organiser_country,startmonth,duration_days,funding_class
0,1496205.00,1496205.00,HORIZON-ERC,False,1,ES,2,1825,low
1,1499150.00,1499150.00,HORIZON-ERC,False,3,ES,9,1826,low
2,1482050.00,1482050.00,HORIZON-ERC,False,1,PL,1,1825,low
3,1495000.00,1495000.00,HORIZON-AG,False,1,DE,4,1825,low
4,1458688.00,1458688.00,HORIZON-ERC,False,1,DE,3,1826,low
...,...,...,...,...,...,...,...,...,...
1995,2457203.50,2717717.50,HORIZON-IA,True,7,AT,9,1095,medium
1996,3998413.00,3998413.00,HORIZON-RIA,False,13,ES,9,1095,medium
1997,4999695.00,4999695.00,HORIZON-RIA,False,18,ES,12,1095,high
1998,3498525.25,3796065.00,HORIZON-RIA,False,9,NO,11,1460,medium


In [179]:
print(processed_unique['funding_class'].value_counts())


funding_class
low       3230
medium    2948
high      2599
Name: count, dtype: int64


In [180]:
processed_unique.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8777 entries, 0 to 8776
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   ecMaxContribution  8777 non-null   float64 
 1   totalCost          8777 non-null   float64 
 2   fundingScheme      8777 non-null   object  
 3   SME                8777 non-null   object  
 4   org_count          8777 non-null   int64   
 5   organiser_country  8777 non-null   object  
 6   startmonth         8777 non-null   int32   
 7   duration_days      8777 non-null   int64   
 8   funding_class      8777 non-null   category
dtypes: category(1), float64(2), int32(1), int64(2), object(3)
memory usage: 523.1+ KB


In [ ]:

def split_train_test(df):
 
    X = df.drop(columns=["funding_class"])
    y = df["funding_class"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=.20, random_state=56, stratify=True
    )
    return X_train, X_test, y_train, y_test


In [ ]:
X_train, X_test, y_train, y_test = split_train_test(processed_unique)

In [ ]:


def remove_outliers_isolation_forest(df):


    num_cols = df.select_dtypes(include='number').columns.tolist()
   
    df_no_na = df.dropna(subset=num_cols)
    
    iso = IsolationForest(contamination=.01, random_state=78)
    mask = iso.fit_predict(df_no_na[num_cols]) == 1  
    return df_no_na.loc[mask].reset_index(drop=True)


In [184]:
X_train = remove_outliers_isolation_forest(X_train)

In [185]:
X_test = remove_outliers_isolation_forest(X_test)

In [186]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6950 entries, 0 to 6949
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ecMaxContribution  6950 non-null   float64
 1   totalCost          6950 non-null   float64
 2   fundingScheme      6950 non-null   object 
 3   SME                6950 non-null   object 
 4   org_count          6950 non-null   int64  
 5   organiser_country  6950 non-null   object 
 6   startmonth         6950 non-null   int32  
 7   duration_days      6950 non-null   int64  
dtypes: float64(2), int32(1), int64(2), object(3)
memory usage: 407.4+ KB


In [187]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1738 entries, 0 to 1737
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ecMaxContribution  1738 non-null   float64
 1   totalCost          1738 non-null   float64
 2   fundingScheme      1738 non-null   object 
 3   SME                1738 non-null   object 
 4   org_count          1738 non-null   int64  
 5   organiser_country  1738 non-null   object 
 6   startmonth         1738 non-null   int32  
 7   duration_days      1738 non-null   int64  
dtypes: float64(2), int32(1), int64(2), object(3)
memory usage: 102.0+ KB


In [188]:
def drop_MaxContribution(df, col='ecMaxContribution'):

    df = df.copy()
    return df.drop(columns=[col])

In [189]:
X_train = drop_MaxContribution(X_train)

In [190]:
X_test = drop_MaxContribution(X_test)

In [191]:
X_train.head()

,totalCost,fundingScheme,SME,org_count,organiser_country,startmonth,duration_days
0,1741737.50,HORIZON-CSA,False,15,MT,10,1095
1,12339067.25,HORIZON-IA,False,30,ES,1,1276
2,1500000.00,HORIZON-ERC,False,1,FI,1,1825
3,10714691.25,HORIZON-IA,False,13,ES,12,1826
4,2826500.00,HORIZON-RIA,False,11,EL,9,1095


In [ ]:

def one_hot_encode_df(df):
  
    df_encoded = df
    
   
    if df_encoded['SME'].dtype == bool:
        df_encoded['SME'] = df_encoded['SME'].astype(int)
    
   
    categorical_cols = ['fundingScheme', 'organiser_country']
    
    
    df_encoded = pd.get_dummies(df_encoded, columns=categorical_cols, drop_first=True)
    
    return df_encoded


In [193]:
X_train = one_hot_encode_df(X_train)
X_test = one_hot_encode_df(X_test)

In [194]:
X_test.head()

,totalCost,SME,org_count,startmonth,duration_days,fundingScheme_EIC,fundingScheme_EIC-ACC,fundingScheme_ERC,fundingScheme_EURATOM-CSA,fundingScheme_EURATOM-IA,...,organiser_country_PL,organiser_country_PT,organiser_country_RO,organiser_country_RS,organiser_country_SE,organiser_country_SI,organiser_country_SK,organiser_country_TR,organiser_country_UK,organiser_country_ZA
0,3944941.38,False,11,3,1279,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
1,1489798.00,False,1,1,1825,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1454635.00,False,1,4,1826,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2788250.00,False,18,10,1095,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2497210.00,False,1,1,1825,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
def standardize_numeric_columns(X_train, X_test, numeric_columns=["totalCost", "org_count", "duration_days"]):
    
    scaler = StandardScaler()
    
    X_train_scaled = X_train
    X_test_scaled = X_test
    
    X_train_scaled[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
    X_test_scaled[numeric_columns] = scaler.transform(X_test[numeric_columns])
    
    return X_train_scaled, X_test_scaled

In [196]:
X_train_stand, X_test_stand = standardize_numeric_columns(X_train, X_test)

In [204]:
counts = pd.Series(y_test).value_counts()
print(counts)

funding_class
low       646
medium    590
high      520
Name: count, dtype: int64
